## **Introduction**

In [ ]:
# Import & Install statements

!pip install tensorflow==2.9.1 --quiet
!pip install transformers==4.21.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
google-cloud-aiplatform 1.64.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.

In [ ]:
# !pip install vaderSentiment

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import transformers
import torch
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.nn.functional import softmax
from tqdm import tqdm

from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from transformers import TFRobertaModel

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

TensorFlow version: 2.9.1
Transformers version: 4.21.0


In [ ]:
# Read the dataset

coffee_df = pd.read_csv('/content/sample_data/coffee_analysis.csv')

In [ ]:
# Take a quick look at the first five rows of the dataset

coffee_df.head()

,name,roaster,roast,loc_country,origin_1,origin_2,100g_USD,rating,review_date,desc_1,desc_2,desc_3
0,“Sweety” Espresso Blend,A.R.C.,Medium-Light,Hong Kong,Panama,Ethiopia,14.32,95,November 2017,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,Flora Blend Espresso,A.R.C.,Medium-Light,Hong Kong,Africa,Asia Pacific,9.05,94,November 2017,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,Ethiopia Shakiso Mormora,Revel Coffee,Medium-Light,United States,Guji Zone,Southern Ethiopia,4.70,92,November 2017,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,Ethiopia Suke Quto,Roast House,Medium-Light,United States,Guji Zone,Oromia Region,4.19,92,November 2017,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,Ethiopia Gedeb Halo Beriti,Big Creek Coffee Roasters,Medium,United States,Gedeb District,Gedeo Zone,4.85,94,November 2017,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [ ]:
'''
Take a more detailed look at important information about the dataset, such as:
    --> The total number of rows & columns
    --> The column names
    --> How many non-null values each column has
    --> Each column's data type
'''

coffee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095 entries, 0 to 2094
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         2095 non-null   object 
 1   roaster      2095 non-null   object 
 2   roast        2080 non-null   object 
 3   loc_country  2095 non-null   object 
 4   origin_1     2095 non-null   object 
 5   origin_2     2095 non-null   object 
 6   100g_USD     2095 non-null   float64
 7   rating       2095 non-null   int64  
 8   review_date  2095 non-null   object 
 9   desc_1       2095 non-null   object 
 10  desc_2       2095 non-null   object 
 11  desc_3       2093 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 196.5+ KB


## **General Preprocessing and Data Cleaning**

In [ ]:
'''
There are three description columns, but we're only going to be using the first one, so the other two get dropped.
But in order to not mess with the original dataset, we store the dataframe without the extra two columns into a new dataframe called coffee_df2.
'''
coffee_df2 = coffee_df.drop(columns = ['desc_2','desc_3']).copy()

In [ ]:
# Check to see if there are any null values in the "new" dataset.
missing_values2 = coffee_df2.isnull().sum()
print(missing_values2)

name            0
roaster         0
roast          15
loc_country     0
origin_1        0
origin_2        0
100g_USD        0
rating          0
review_date     0
desc_1          0
dtype: int64


In [ ]:
'''
Since there are 15 rows where the roast is null, we need to impute them. But the specific roast of a coffee blend is dependent on the roaster; not every roaster is going to roast their coffee the same way.
So first, we'll look at the roast distribution, i.e., for the 4 roasters associated with the 15 null value rows, how do they roast their coffee?
'''

null_rows_roast = coffee_df2[coffee_df2['roast'].isnull()]

roasters_with_missing_roasts = null_rows_roast['roaster'].unique()

roasters_with_null_roast_df = pd.DataFrame(roasters_with_missing_roasts, columns = ['Roasters with missing roasts'])

roast_distribution = coffee_df2[coffee_df2['roaster'].isin(roasters_with_missing_roasts)].groupby(['roaster', 'roast']).size().unstack(fill_value = 0)

print(roasters_with_null_roast_df)

  Roasters with missing roasts
0     Barefoot Coffee Roasters
1            Difference Coffee
2               Collage Coffee
3       Bonfire Coffee Company
4                    GK Coffee
5              Taster's Coffee
6                Port of Mokha


In [ ]:
roasters_with_null_roast_df

,Roasters with missing roasts
0,Barefoot Coffee Roasters
1,Difference Coffee
2,Collage Coffee
3,Bonfire Coffee Company
4,GK Coffee
5,Taster's Coffee
6,Port of Mokha


In [ ]:
print(f"Roast Distribution: \n{roast_distribution}\n")

Roast Distribution: 
roast              Light  Medium  Medium-Light
roaster                                       
Collage Coffee         1       0             4
Difference Coffee      0       0             1
GK Coffee              7       5            22
Taster's Coffee        3       0             0



In [ ]:
'''
Based on the roast distribution above, the imputation breakdown is as follows:
    --> Collage Coffee, GK Coffee, & Difference Coffee each tend to give their coffees a medium-light roast, so we'll impute null values associated with those 3 roasters with Medium-Light.
    --> But Taster's Coffee tends to give their coffee a light roast, so we'll impute null values associated with them with Light.
    --> For the other three, because they don't have any other rows with the roast listed, we have to tackle them separately below.
'''

coffee_df2.loc[(coffee_df2['roaster'] == 'Collage Coffee') & (coffee_df2['roast'].isnull()), 'roast'] = 'Medium-Light'
coffee_df2.loc[(coffee_df2['roaster'] == 'GK Coffee') & (coffee_df2['roast'].isnull()), 'roast'] = 'Medium-Light'
coffee_df2.loc[(coffee_df2['roaster'] == 'Difference Coffee') & (coffee_df2['roast'].isnull()), 'roast'] = 'Medium-Light'

coffee_df2.loc[(coffee_df2['roaster'] == 'Taster\'s Coffee') & (coffee_df2['roast'].isnull()), 'roast'] = 'Light'

remaining_null_values = coffee_df2['roast'].isnull().sum()
print(f"Remaining null values in roast: {remaining_null_values}")

Remaining null values in roast: 4


In [ ]:
# Since a coffee's roast is also dependent on the region of origin, let's look at the two origin columns, and impute based on that.

missing_roasters_rows = coffee_df2[(coffee_df2['roaster'].isin(['Barefoot Coffee Roasters', 'Bonfire Coffee Company', 'Port of Mokha'])) & (coffee_df2['roast'].isnull())]
print(missing_roasters_rows[['roaster', 'origin_1', 'origin_2']])

                       roaster         origin_1            origin_2
209   Barefoot Coffee Roasters        Guatemala       Not Disclosed
815     Bonfire Coffee Company       Jimma Zone        Oromia State
1484             Port of Mokha  Al Hayma Region  Sana’A Governorate
1486             Port of Mokha   Al Wadi Region  Sana’A Governorate


In [ ]:
# Start off by checking all the rows where the origin combinations above are used.

origin_combinations = [
    ('Guatemala', 'Not Disclosed'),
    ('Jimma Zone', 'Oromia State'),
    ('Al Hayma Region', 'Sana’ Governorate'),
    ('Al Wadi Region', 'Sana’A Governorate')
]

filtered_rows = coffee_df2[
    coffee_df2[['origin_1', 'origin_2']].apply(tuple, axis = 1).isin(origin_combinations)
]

print(filtered_rows)

                                                   name  \
209                                           Cold Brew   
342                                         Nano Challa   
363                                Bombe Natural Sidama   
395                                Nano Challa Ethiopia   
561   Ethiopia Sidama Naia Bomb Natural Vertical Res...   
573                   Ethiopia Limu Nano Challa Lot #10   
581   Ethiopia Sidama Naia Bombe Dry Ferment Vertica...   
720                       Ethiopia Sidama Bombe Natural   
815                      Ethiopia Nano Challa Cold Brew   
869                               Limu Ethiopia Natural   
1486                                      Yemen Al Wadi   

                            roaster         roast    loc_country  \
209        Barefoot Coffee Roasters           NaN  United States   
342       Bird Rock Coffee Roasters  Medium-Light  United States   
363               Broadsheet Coffee  Medium-Light  United States   
395                

In [ ]:
# Then impute based on the most common roast type that makes sense for those three roasters based on their origin combinations.

coffee_df2.loc[(coffee_df2['roaster'] == 'Barefoot Coffee Roasters') & (coffee_df2['roast'].isnull()), 'roast'] = 'Medium-Light'
coffee_df2.loc[(coffee_df2['roaster'] == 'Bonfire Coffee Company') & (coffee_df2['roast'].isnull()), 'roast'] = 'Medium'
coffee_df2.loc[(coffee_df2['roaster'] == 'Port of Mokha') & (coffee_df2['roast'].isnull()), 'roast'] = 'Medium-Light'

In [ ]:
# One final check to see if there are any null values still left.
missing_values3 = coffee_df2.isnull().sum()
print(missing_values3)

name           0
roaster        0
roast          0
loc_country    0
origin_1       0
origin_2       0
100g_USD       0
rating         0
review_date    0
desc_1         0
dtype: int64


In [ ]:
# Since underscores are easier for Python to parse then dashes, let's replace them. First check which values in the 'roast' column have dashes.

unique_roasts = coffee_df2['roast'].unique()

print(unique_roasts)

['Medium-Light' 'Medium' 'Light' 'Medium-Dark' 'Dark']


In [ ]:
# Then replace the dashes and check again to confirm.

coffee_df2['roast'] = coffee_df2['roast'].str.replace('-', '_')

updated_roasts = coffee_df2['roast'].unique()

print(updated_roasts)

['Medium_Light' 'Medium' 'Light' 'Medium_Dark' 'Dark']


In [ ]:
# Next change the review date column to a datetime format.

coffee_df2['review_date'] = pd.to_datetime(coffee_df2['review_date'], errors = 'coerce')

<ipython-input-21-a58f55420c8a>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  coffee_df2['review_date'] = pd.to_datetime(coffee_df2['review_date'], errors = 'coerce')


In [ ]:
# Remove the time portion of the modified review date column values

coffee_df2['review_date'] = coffee_df2['review_date'].dt.date

coffee_df2['review_date'] = coffee_df2['review_date'].astype(str)

In [ ]:
# Check the modified review date column to confirm that the modifications have worked

date_null_values = coffee_df2['review_date'].isnull().sum()

print(coffee_df2['review_date'].unique())

['2017-11-01' '2017-12-01' '2018-01-01' '2018-02-01' '2018-03-01'
 '2018-04-01' '2018-05-01' '2018-06-01' '2018-07-01' '2018-08-01'
 '2018-09-01' '2018-10-01' '2018-11-01' '2018-12-01' '2019-02-01'
 '2019-01-01' '2019-03-01' '2019-04-01' '2019-05-01' '2019-06-01'
 '2019-07-01' '2019-08-01' '2019-09-01' '2019-10-01' '2019-11-01'
 '2019-12-01' '2020-01-01' '2020-02-01' '2020-03-01' '2020-04-01'
 '2020-05-01' '2020-06-01' '2020-07-01' '2020-08-01' '2020-09-01'
 '2020-10-01' '2020-11-01' '2020-12-01' '2021-01-01' '2021-02-01'
 '2021-03-01' '2021-04-01' '2021-05-01' '2021-06-01' '2021-07-01'
 '2021-08-01' '2021-09-01' '2021-10-01' '2021-11-01' '2021-12-01'
 '2022-01-01' '2022-02-01' '2022-03-01' '2022-04-01' '2022-05-01'
 '2022-06-01' '2022-07-01' '2022-08-01' '2022-09-01' '2022-10-01'
 '2022-11-01']


## **Text Cleaning and NLP**

In [ ]:
'''
Create a function that does the following to the text in the desc_1 column when it's called on coffee_df2:
    --> Lowers the text.
    --> Gets rid of punctuation marks.
    --> Tokenizes the text.
    --> Removes Stop Words
    --> Lemmatizes the text.
    --> Does a general cleaning of the text to finish up.
'''

def clean_text(text):
  text = text.lower()

  text = text.translate(str.maketrans('', '', string.punctuation))

  tokens = word_tokenize(text)

  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]

  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  cleaned_text = ' '.join(tokens)

  return cleaned_text

In [ ]:
coffee_df2['cleaned_desc_1'] = coffee_df2['desc_1'].apply(clean_text)

print(coffee_df2[['desc_1', 'cleaned_desc_1']].head())

                                              desc_1  \
0  Evaluated as espresso. Sweet-toned, deeply ric...   
1  Evaluated as espresso. Sweetly tart, floral-to...   
2  Crisply sweet, cocoa-toned. Lemon blossom, roa...   
3  Delicate, sweetly spice-toned. Pink peppercorn...   
4  Deeply sweet, subtly pungent. Honey, pear, tan...   

                                      cleaned_desc_1  
0  evaluated espresso sweettoned deeply rich choc...  
1  evaluated espresso sweetly tart floraltoned ho...  
2  crisply sweet cocoatoned lemon blossom roasted...  
3  delicate sweetly spicetoned pink peppercorn da...  
4  deeply sweet subtly pungent honey pear tangeri...  


In [ ]:
# Create a Vectorizer & Document-Term Matrix, and then run Latent Dirichlet Allocation on the Document-Term Matrix.


vectorizer = CountVectorizer(max_df = 95, min_df = 2, stop_words = 'english')
doc_term_matrix = vectorizer.fit_transform(coffee_df2['cleaned_desc_1'])

lda = LatentDirichletAllocation(n_components = 5, random_state = 42)
lda.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [ ]:
# Create a function that displays the most common topics after having run Latent Direchlet Allocation on the Document-Term Matrix.

def display_topics(model, feature_names, num_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

num_top_words = 10
display_topics(lda, vectorizer.get_feature_names_out(), num_top_words)

Topic #1:
peppercorn pomegranate wisteria lime pistachio peach brittle mulberry citrus frankincense
Topic #2:
marjoram lychee bergamot blueberry grape blossom amber barrel thyme brightly
Topic #3:
lavender mango pear brown musk pungent layered hop fine quietly
Topic #4:
powder strawberry tamarind nougat freesia velvetysmooth tea wild rose blackberry
Topic #5:
tobacco lily scorched pipe raisin fudge cinnamon apple spice fresh


## **Sentiment Analysis with VADER**

In [ ]:
# Initialize a VADER Sentiment Intensity Analyzer object, and then run Sentiment Analysis on the preprocessed dataset.

vader = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
  return vader.polarity_scores(text)
coffee_df2['sentiment_scores'] = coffee_df2['cleaned_desc_1'].apply(get_sentiment_score)
coffee_df2['compound'] = coffee_df2['sentiment_scores'].apply(lambda x: x['compound'])

In [ ]:
'''
VADER by default gives Sentiment Analysis scores that range from -1 to 1.
But to make those results a little easier to parse & understand, this function scales the results to both a 0-1 scale & a 1-10 scale.
It then applies both scalings to the sentiment analysis scores that we already have, and does that by creating a column for each scaling.
'''

def scale_scores(compound_score):
  scaled_0_1 = (compound_score + 1)/2
  scaled_1_10 = scaled_0_1 * 9 + 1
  return scaled_0_1, scaled_1_10

coffee_df2[['compound_scaled_0_1', 'compound_scaled_1_10']] = coffee_df2['compound'].apply(lambda x: pd.Series(scale_scores(x)))
coffee_df2[['cleaned_desc_1', 'sentiment_scores', 'compound', 'compound_scaled_0_1', 'compound_scaled_1_10']].head(20)

,cleaned_desc_1,sentiment_scores,compound,compound_scaled_0_1,compound_scaled_1_10
0,evaluated espresso sweettoned deeply rich choc...,"{'neg': 0.0, 'neu': 0.84, 'pos': 0.16, 'compou...",0.8310,0.91550,9.23950
1,evaluated espresso sweetly tart floraltoned ho...,"{'neg': 0.0, 'neu': 0.711, 'pos': 0.289, 'comp...",0.9217,0.96085,9.64765
2,crisply sweet cocoatoned lemon blossom roasted...,"{'neg': 0.0, 'neu': 0.754, 'pos': 0.246, 'comp...",0.8442,0.92210,9.29890
3,delicate sweetly spicetoned pink peppercorn da...,"{'neg': 0.0, 'neu': 0.787, 'pos': 0.213, 'comp...",0.7430,0.87150,8.84350
4,deeply sweet subtly pungent honey pear tangeri...,"{'neg': 0.0, 'neu': 0.79, 'pos': 0.21, 'compou...",0.8169,0.90845,9.17605
5,delicate richly sweetly tart dried hibiscus fi...,"{'neg': 0.0, 'neu': 0.647, 'pos': 0.353, 'comp...",0.9041,0.95205,9.56845
6,hightoned floral dried apricot magnolia almond...,"{'neg': 0.045, 'neu': 0.955, 'pos': 0.0, 'comp...",-0.1280,0.43600,4.92400
7,delicate sweetly savory lemon verbena allspice...,"{'neg': 0.0, 'neu': 0.785, 'pos': 0.215, 'comp...",0.7506,0.87530,8.87770
8,hightoned crisply sweet lemon blossom apricot ...,"{'neg': 0.0, 'neu': 0.824, 'pos': 0.176, 'comp...",0.7184,0.85920,8.73280
9,balanced sweetsavory red currant roasted cacao...,"{'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'comp...",0.7579,0.87895,8.91055


In [ ]:
# Finally, save the results to a CSV file.
coffee_df2.to_csv('coffee_VADER_sentiment_analysis_results.csv', index = False)